# Homework1: Earthquake Occurrence Statistics

The statistics of earthquake occurrence is revealed from catalogs of seismicity, which include event time, location and magnitude. We will talk about how earthquakes are located and how magnitudes are estimated separately, but for now it is sufficient to know that this information can be easily acquired. With such catalogs it is possible to compare the seismic activity of different regions, make informed assessments about the frequency of earthquake occurrence, and learn about the fault rupture process. Maps of the earthquakes in catalogs over time reveal the structure of faulting in a region, and provide a framework with which to study the seismotectonics of a region.

There are two primary earthquake statistics used by seismologists. They are the Gutenberg-Richter relationship (Gutenberg and Richter, 1949), and the Omori Law (Omori, 1894). 

Gutenberg and Richter found that when the logarithm of the number of earthquakes is plotted vs. magnitude that the distribution (data) may be described by linear model, log(N)=A+Bm, where N is the number of earthquakes equal to or larger than m, m is the magnitude and A (y-intercept) and B (slope) are refered to as the Gutenberg-Richter statistics or coefficients. They found that on a global scale, and subsequently more generally, the B-value or the slope of the Gutenberg-Richter line is approximately equal to -1. Thus for each increase in earthquake magnitude there are approximately 10 times fewer earthquakes. If, for example, there are 100 M3 events in a region per year, then the Gutenberg-Richter relationship generally finds that there would be approximately 10 M4 events and 1 M5 event in each year. For magnitudes larger than M5 there would be fewer than one event per year. Gutenberg-Richter is a very important earthquake statistic because it is used to determine the rates of earthquake occurrence, which is a key step in characterizing earthquake hazards (we will see this in future homework exercises). 

The Omori Law is used to characterize the rate at which aftershocks occur following a large mainshock event. This statistic is used for comparing the aftershock productivity of different earthquakes and regions, make forecasts of the likelihood of large damaging aftershocks and to distinguish between earthquake faulting and possibly geothermal or volcanic-related seismicity by examining whether the distribution describes a "mainshock/aftershock" pattern or is more "swarm-like". 

In this homework you will use python code in this notebook to investigate the Gutenberg-Richter and Omori statistics for the San Francisco Bay Area, as well as develop numerical analysis skills using python. 

Note: This is not a python class, but the primary programming tool that we will be used is python. However, if you know MatLab or have other programing background and would prefer to use it, you are free to use those tools instead.

In [ ]:
#Initial Setup and Subroutine Definitions - No need to edit this cell

import math
import datetime
import numpy as np
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader as cReader
import pandas as pd

#The following are useful functions/subroutines

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two geographic points
    on the earth (specified in decimal degrees)

    All args must be of equal length.
    
    The first pair can be singular and the second an array

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371.0 * c
    return km

def parseCatalog(p):
    '''
    Function to slice an ANSS catalog loaded as a pandas dataframe and return arrays of info, including a days array
    '''
    year=p['DateTime'].dt.year
    month=p['DateTime'].dt.month
    day=p['DateTime'].dt.day
    hour=p['DateTime'].dt.month
    minute=p['DateTime'].dt.minute
    sec=p['DateTime'].dt.second
    lat=p['Latitude'].values
    lon=p['Longitude'].values
    mag=p['Magnitude'].values

    days = countDays(len(year),year,month,day)
    return year,month,day,hour,minute,sec,lat,lon,mag,days

def countDays(npts,y,m,d):
    '''
    Function to create an array of days
    '''
    days=np.zeros(npts)
    for i in range(0,npts,1):
        d0 = datetime.date(y[0], m[0], d[0])
        d1 = datetime.date(y[i], m[i], d[i])
        delta = d1 - d0
        days[i]=delta.days
    return days


## The Catalog
We have downloaded the Advanced National Seismic System (ANSS) catalog from 1900 to 2020 for you to use. This catalog has all events in the aforementioned time range located within 100 km of UC Berkeley. Columns of this catalog include information about the catalogued earthquakes, including the date and time of each event, its location in latitude, longitude and depth, and the event magnitude.  

The following python code reads this catalog file and places the information in arrays for analysis.

You can get catalogs from various sources such as:

    NCEDC: https://www.ncedc.org/ncedc/catalog-search.html

    USGS: https://www.usgs.gov/programs/earthquake-hazards/earthquakes

In [ ]:
# read data
bay_catalog=pd.read_csv('data/bay_area_anss_1900_2020.csv')


#There are NaN in magnitude - lets purge those
bay_catalog=bay_catalog.dropna(subset=['Magnitude'])  #this drops the NaN from the Magnitude columns
bay_catalog=bay_catalog.reset_index(drop=True)        #this resets the index for the updated pandas dataframe


bay_catalog['DateTime'] = pd.to_datetime(bay_catalog['DateTime'])  #this converts the format to datetime which has some nice functions


bay_catalog.head()

In [ ]:
#create data arrays - take a look at the function parseCatalog() above to see what it is doing
year,month,day,hour,minute,sec,lat,lon,mag,days = parseCatalog(bay_catalog)

# Exercise 1: Explore the raw catalog (10 pts)


## Print basic catalog info
Find the number of events, the number of days from the first event, the minimum magnitude, and the maximum magnitude. Use the variable names provided so that the example print statements will work.

In [ ]:
#Finish this code
nevt = ...
ndays = ... 
min_mag = ... 
max_mag = ... 

#Example print statements
#print(f'There are {nevt} events in the catalog.')
#print(f'The first event was {ndays:.0f} days before the last.')
#print(f'The magnitudes range from {min_mag} to {max_mag}.')

## Plot the catalog time series
Make an x-y scatter plot showing the magnitude of the earthquake on the y-axis and the time of the event on the x-axis. For this it is convenient that the last output of the `readAnssCatalog` function is the number of days since the beginning of the catalog. The plot will show that the catalog is not uniform due to the fact that over time as more seismic recording stations were installed more earthquakes could be detected and properly located.


In [ ]:
#Now the plot
#These are the basic plotting commands - there are many more functions and features that can be used
plt.plot(days, mag,'o',alpha=0.2,markersize=5)
plt.xlabel('Days')
plt.ylabel('Magnitude')
plt.title('Raw Event Catalog')
plt.grid()
plt.show()

### Map the Raw Earthquake Catalog

On a map of the Bay Area plot the location of events in the declustered catalog. Scale the marker color and size by magnitude. The initial map with faults plotted is below. To plot the seismicity you will want to use `plt.scatter()` and `plt.plot()` calls. Note that if you type `plt.scatter?` in a cell you will get the documentation for the function.


In [ ]:
#First read in the fault data - No need to edit this cell

a=pd.read_table('data/Hayward.txt') 
hay_lon=a['x'].values
hay_lat=a['y'].values

a=pd.read_table('data/San_Andreas.txt') 
SA_lon=a['x'].values - 0.03 #adjustment to the west
SA_lat=a['y'].values

a=pd.read_table('data/San_Gregorio.txt') 
SG_lon=a['x'].values
SG_lat=a['y'].values

a=pd.read_table('data/Calaveras.txt') 
cal_lon=a['x'].values
cal_lat=a['y'].values

a=pd.read_table('data/Hunting_Creek.txt') 
HC_lon=a['x'].values
HC_lat=a['y'].values

a=pd.read_table('data/Rodgers_Creek.txt') 
RC_lon=a['x'].values
RC_lat=a['y'].values

a=pd.read_table('data/Concord.txt') 
con_lon=a['x'].values
con_lat=a['y'].values

a=pd.read_table('data/Greenville.txt') 
grn_lon=a['x'].values
grn_lat=a['y'].values

a=pd.read_table('data/Maacama.txt') 
m_lon=a['x'].values
m_lat=a['y'].values

In [ ]:
#Make a Map of the earthquake catalog

# Set Corners of Map
lat0=37.0
lat1=38.75
lon0=-123.25
lon1=-121.5
tickstep=0.25 #for axes
latticks=np.arange(lat0,lat1+tickstep,tickstep)
lonticks=np.arange(lon0,lon1+tickstep,tickstep)

# coordinates for UC Berkeley
Berk_lat = 37.8716
Berk_lon = -122.2727

plt.figure(1,(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([lon0, lon1, lat0, lat1], crs=ccrs.PlateCarree())
ax.coastlines(resolution='10m',linewidth=1)
ax.set_xticks(lonticks)
ax.set_yticks(latticks, crs=ccrs.PlateCarree())
ax.set(xlabel='Longitude', ylabel='Latitude',title='Raw Catalog')

#Sort Descending to plot largest events on top - finish the code
indx=np.argsort(mag)   #determine sort index
x=...            #apply sort index
y=...            #apply sort index
z=...            #exponent to scale size

#Modify the plt.scatter() function to plot the earthquakes
#plt.scatter(...)


plt.plot(hay_lon,hay_lat,'-',color='red',linewidth=2,label='Hayward Fault')
plt.plot(SA_lon,SA_lat,'-',color='orange',linewidth=2,label='San Andreas Fault')
plt.plot(SG_lon,SG_lat,'-',color='yellow',linewidth=2,label='San Gregorio Fault')
plt.plot(cal_lon,cal_lat,'-',color='green',linewidth=2,label='Calaveras Fault')
plt.plot(con_lon,con_lat,'-',color='cyan',linewidth=2,label='Concord Fault')
plt.plot(grn_lon,grn_lat,'-',color='lime',linewidth=2,label='Greenville Fault')
plt.plot(m_lon,m_lat,'-',color='blueviolet',linewidth=2,label='Maacama Fault')
plt.plot(RC_lon,RC_lat,'-',color='magenta',linewidth=2,label='Rodgers Creek Fault')
plt.plot(HC_lon,HC_lat,'-',color='blue',linewidth=2,label='Hunting Creek Fault')
plt.plot(Berk_lon,Berk_lat,'rs',markersize=8,label='UC Berkeley')  # plot red square on Berkeley Campus
plt.legend()
plt.show()

**QUESTIONS**:

1. Describe the seismicity and any patterns that you see.

2. How well does the seismicity show the region's major faults?

3. Reviewing fault maps online what faults missing from the plot might be added to better explain the seismicity?

#### answers here

# Exercise 2: Compute the Gutenberg-Richter statitistics (30 pts)

Follow the steps below to compute the Gutenberg Richter statistics for the raw catalog.

- Define a magnitude bin array (m) using `np.arange()`. Considering m from 0 to the maximium magnitude in the catalog with 0.1 steps will be sufficient.

- Next,  write a loop that counts all events greater than equal to a given magnitude bin, and places it in a array, N, with the same length as m. You can use the `np.count_nonzero()` function for this.

- Commonly the the number of events in each bin, N, is divided by the number of years the catalog spans to give the annual number of events per year. The result is the annual number of events for each magnitude (m) bin.

- Then lake the `np.log10()` of N and for clarity you can name the array logN.

- Plot logN vs. m, where LogN is on the y-axis and m is on the x-axis.

- The Gutenberg-Richter relationship is linear in logN - m space. Does the catalog data agree with this assumption? Is there a range of m where there is better agreement with the linear model?

- What is the reason for the different trend at small magnitude?

#### Calculate the Gutenberg-Richter data

In [ ]:
# define a range of mag bins with width 0.1 magnitude per bin
m=np.arange(...) 

# Preallocate the vector logN with a zeros vector of size len(m)
logN = np.zeros(len(m))
ny = (max(days)-min(days))/365 

# Find N
for i in range(0,len(m),1):
    logN[i]= np.log10(np.count_nonzero(...) 



Make a plot of the distribution.

In [ ]:

plt.figure()
#code for plot here
plot.show()

## Fit the data to find the Gutenberg Richter statistics.

Now, fit the data with the Gutenberg Richter relationship $log_{10}(N(m))$=A+Bm. In other words, "invert" the data to find the applied model parameters. Since the model is linear we can use one of the methods to fit a line to the data. `np.polyfit()` is suitable function for this.

To get better results limit the m, logN arrays to consider only the portion of the data that has a linear behavior, and compare with the data. `np.polyval()` can be used to create the model series. You can overlay the modeled line with the data either in a single `plt.plot()` function, or through sequential calls to the function.

In [ ]:
#Compute the model fit using np.polyfit() and use np.polyval() to generate the model points (yfit)
x=m[..]
y=logN[..]
p=np.polyfit(...,...,1)
yfit=np.polyval(...,...)

#Plot comparing input data as points and model as a red line

#Print model coefficients
print(f'A={p[1]:0.3f} B={p[0]:0.3f}')

#### Questions

1. Discuss the quality of the fit of the Gutenberg-Richter model.

2. Given the Gutenberg-Richter parameters you found, what is the annual frequncy of M5, 6 and 7 earthquakes. What is the average recurrence (interval) time of these earthquakes?

In [ ]:
#Put your work to estimate annual frequency and recurrence intervals here

# Exercise 3: Declustering (30 pts)

In the above analysis mainshocks (primary events) and aftershocks are mixed together. The results for the Gutenberg-Richter statistics were generally pretty good, however a correct implementation of Gutenberg-Richter considers only the primary events. Therefore, we seek a catalog with aftershocks removed in order to improve our assessment of the Gutenberg-Richter statistics. The process to remove aftershocks is called declustering.

In this exercise, you will evaluate a published declustering method as you use it to decluster the catalog analyzed above. Then you will re-compute the Gutenberg-Richter coefficients for the declustered catalog in order to examine the affect on the G-R statistics.


## Declustering Algorithm

The analysis that was just performed was for the raw catalog, which means that it includes all events. However Gutenberg-Richter is really interested in the occurrence of primary "main shock" events, and therefore it is necessary to decluster the catalog to obtain an unbiased estimate of the G-N coefficients. Declustering here means remove the aftershocks from the catalog. This is done using an algorithm that relates the "expected" time and distance range of aftershocks from a given mainshock. Large mainshocks will result in aftershock populations that, statistically speaking, have a greater likelihood to occur over longer time periods and greater mainshock-aftershock distances compared with smaller mainshock-aftershock series. 

The code block below defines a declustering algorithm. This algorithm uses distance and time metrics that are magnitude dependent, called 'Dtest' and 'Ttest'. If a given event falls within the maximal values defined by Dtest and Ttest for its magnitude it is deemed an aftershock and removed from the catalog. After all events are processed, the remaining catalog is then comprised of only primary events. This declustered catalog can be used to estimate more accurate Gutenberg-Richter statistics. Furthermore, we can study the aftershock events that the algorithm removed for a given earthquake in the context of the Omori Law statistics (Exercise 4).

Because aftershock identification is an empirical procedure, there are many different ways to define the Dtest and Ttest relationships. Stiphout et al., (2012, on page 10) summarizes three different definitions of the Dtest/Ttest relationships originally proposed by Uhrhammer (1986), Knopoff and Gardner (1972). The following are the equations in Stiphout et al. (2012) are for method 1 in the declustering function defined below.
<img src="./Figures/window_approx.png">

Compare the event reduction rate (final number divided by the initial number of events) for the three different proposed distance and time windows. You can do this by adding a logical (if statement) tree to enable switching between different definitions of Dtest and Ttest in declustering_algorithm below. The first definition (Eqn 1 from Stiphout et al., 2012, p.10) has already been completed. Note that some of these algorithms will not run instantaneously and might take up to a few minutes.

In [ ]:
def declustering_algorithm(cat, definition=1):
    '''
    Decluster a catalog
    
    note: This function may take a few minutes to complete
    
    calls parseCatalog()
    
    Inputs: 
    
    cat must be an anss formatted pandas datafram
    definition is the algorithm (1 - 2) from Stiphout, 2012, which determines Dtest and Ttest values
        Definition = 1 : Gardner and Knopoff, 1974 [default]
        Definition = 2 : Uhrhammer, 1986
    
    '''
    
    # do not edit
    cnt=0
    save=np.zeros((1,10000000),dtype=int)

    # grab catalog arrays
    year,month,day,hour,minute,sec,lat,lon,mag,days = parseCatalog(cat)
    ne=len(year)

    # main for-loop over events
    for i in range(0,ne,1):
        
        if definition == 1:
            
            # Definition #1 : Knopoff and Gardner, 1972
            Dtest=np.power(10,0.1238*mag[i]+0.983)
            if mag[i] >= 6.5:
                Ttest=np.power(10,0.032*mag[i]+2.7389)
            else:
                Ttest=np.power(10,0.5409*mag[i]-0.547)
  
        elif definition == 2:

            # Definition #2 : Uhrhammer, 1986
            Dtest=np.exp(-1.204+0.804*mag[i]) # SOLUTION
            Ttest=np.exp(-2.87+1.235*mag[i]) # SOLUTION

            
            
        a=days[i+1:ne]-days[i]
        m=mag[i+1:ne]
        b=haversine_np(lon[i],lat[i],lon[i+1:ne],lat[i+1:ne])

        icnt=np.count_nonzero(a <= Ttest)
        if icnt > 0:
            itime=np.array(np.nonzero(a <= Ttest)) + (i+1)
            for j in range(0,icnt,1):             
                if b[j] <= Dtest and m[j] < mag[i]:
                    save[0][cnt]=itime[0][j]
                    cnt += 1 # save contains index of aftershocks in cat

    #Note this is an array of indexes that will be used to delete events flagged 
                        #as aftershocks
    save=np.delete(np.unique(save),0)  
    
    # Filter or slice out the declustered and aftershock dataframe catalogs from the 
    # original dataframe catalog "data" using "save" above.
    cat_aftershocks = cat.iloc[np.unique(save)] 

    cat_declustered = cat.iloc[~cat.index.isin(save)]
    
    cat_aftershocks.reset_index(drop=True, inplace=True)
    cat_declustered.reset_index(drop=True, inplace=True)
    
    return cat_declustered, cat_aftershocks

In [ ]:
# Run the declustering algorithm
method = 1
data=bay_catalog
data_declustered, data_aftershocks = declustering_algorithm(data, definition=method)

# This condition should print out "True" if the catalogs were separated correctly
# The sum of the aftershocks plus the primary events should equal the length of the original catalog

#The sum of the aftershocks and the primary events should equal the original length of the data. The following line should return "True"
len(data) == len(data_aftershocks) + len(data_declustered)

## Plot a map showing the declustered catalog 

In [ ]:
#First set variables for declustered catalog using parseCatalog()
year,month,day,hour,minute,sec,lat,lon,mag,days = parseCatalog(data_declustered)

#Code for map here


## Re-compute the Gutenberg-Richter statistics as above for the declustered catalog


In [ ]:
#Code comparing the Gutenberg-Richter parameters, and the fit to the data.

### Questions

- How many events were removed from the catalog with each declustering algorithm?

- How do the Gutenberg-Richter A and B values differ using the two declustering algorithms?

- Compare the estimated recurrence intervals for M6 and M7 earthquakes.

- Compare your estimated recurrence values with what has been presented in the USGS Earthquake Hazard Assessments for the return period for Hayward fault earthquakes. The Hayward fault data can be found from https://www.usgs.gov/news/hayward-fault-it-due-a-repeat-powerful-1868-earthquake?qt-news_science_products=3#qt-news_science_products

# Exercise 4: Omori Law for Loma Prieta M6.9 Event (30 pts)

Here we will use the declustering algorithm to identify aftershocks of the October 18 1989 at 04:15am (October 17 at 5:15pm PDT) the M6.9 Loma Prieta earthquake occurred in the Santa Cruz mountains approximately 80 km southwest of the Berkeley Campus. This wiki has some background information for the earthquake: https://en.wikipedia.org/wiki/1989_Loma_Prieta_earthquake

### Load the Earthquake Catalog



In [ ]:
# read data

LP_catalog=pd.read_csv('data/bay_area_anss_1989_1995.csv')

LP_catalog=LP_catalog.dropna(subset=['Magnitude'])  #this drops the NaN from the Magnitude columns
LP_catalog=LP_catalog.reset_index(drop=True)

#the following function just reformats to a DateTime object format
LP_catalog['DateTime'] = pd.to_datetime(LP_catalog['DateTime'])
LP_catalog.head()

#parse catalog into arrays
year,month,day,hour,minute,sec,lat,lon,mag,days = parseCatalog(LP_catalog)
LP_catalog

### Map the LP catalog

In [ ]:
lat0=36.0
lat1=38.5
lon0=-123.0
lon1=-121.0
tickstep=0.5 #for axes
latticks=np.arange(lat0,lat1+tickstep,tickstep)
lonticks=np.arange(lon0,lon1+tickstep,tickstep)

# coordinates for UC Berkeley
Berk_lat = 37.8716
Berk_lon = -122.2727

plt.figure(1,(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([lon0, lon1, lat0, lat1], crs=ccrs.PlateCarree())
ax.coastlines(resolution='10m',linewidth=1)
ax.set_xticks(lonticks)
ax.set_yticks(latticks)
ax.set(xlabel='Longitude', ylabel='Latitude',title='Raw Catalog')

# Sort by magnitude to plot largest events on top
LP_catalog_sorted = LP_catalog.sort_values(by=['Magnitude'])
#exponent to scale marker size
z=np.exp(LP_catalog_sorted['Magnitude'])    

plt.scatter(LP_catalog_sorted['Longitude'], LP_catalog_sorted['Latitude'], s=z, 
            c=LP_catalog_sorted['Magnitude'], cmap='plasma',alpha=0.4,marker='o') # plot circles on EQs
plt.plot(Berk_lon,Berk_lat,'s',color='limegreen',markersize=8)  # plot red square on Berkeley Campus
plt.colorbar(label='Magnitude')

plt.show()

#### Parse LP_catalog for 3 months from the earthquake & create arrays

In [ ]:
#You can parse the catalog in different ways. Using boolean logic in a series of filters, or more simply making use
#of the pandas between function, e.g. dataframe.between(). The example below filters 4 months of data beginning at
#the time of the mainshock

#First lets make a deep copy of the data
LPEQ=LP_catalog.copy(deep=True)

start_date=pd.datetime(1989,10,18,0,4)
end_date=pd.datetime(1990,2,18,0,0)
LPEQ=LPEQ[LPEQ['DateTime'].between(start_date,end_date)]
LPEQ=LPEQ.reset_index(drop=True)

year,month,day,hour,minute,sec,lat,lon,mag,days = parseCatalog(LPEQ)

## Plot the Loma Preita time series


In [ ]:
# plot magnitude vs. days from mainshock


## Plot the Loma Preita Earthquake Catalog in map view


In [ ]:
#Code for LP sequence map here

## Decluster the Raw Catalog for the Loma Prieta time period

We use the same decluster algorithm previously to identify aftershocks and remove them from the 30-day Loma Preita catalog.


In [ ]:
#declustering code here
data_dec, data_after = declustering_algorithm(...,...)

# This condition should print out "True" if the catalogs were separated correctly
len(LPEQ) == len(data_after) + len(data_dec)

Create two new sets of event info arrays, one for the declustered catalog and one for the aftershock catalog.

In [ ]:
dyear,dmonth,dday,dhour,dminute,dsec,dlat,dlon,dmag,ddays = parseCatalog(data_dec)
ayear,amonth,aday,ahour,aminute,asec,alat,alon,amag,adays = parseCatalog(data_after)
dnevt =len(ddays)
anevt=len(adays)

In [ ]:
#Plot Aftershock Catalog in time series


## Plot the declustered Loma Prieta mainshock catalog in map view


In [ ]:
#Make a Map of the declustered events



## Plot the declustered Loma Prieta aftershock catalog in map view


In [ ]:
#Make a Map of Aftershock events


## Omori statistics

To compute the Omori statistics we want to bin the log10 of the number of aftershocks each day following the mainshock and fit a power law equation such as:

$$
\begin{matrix}
N=\frac{A}{(t+\epsilon)^P}, 
\end{matrix}
$$

where t is time in days, N is the number of earthquakes in the 24 hour period, and $\epsilon$ is a small number (fraction of a day) to avoid the singularity at zero time. A and P are the coeffients that we want to find through regression. This power law equation can be linearized by simply taking the log10 of both sides giving:

$$
\begin{matrix}
log_{10}(N)=a + P*log_{10}(t+\epsilon)
\end{matrix}
$$

Note: a=log10(A). Also when we regress the data, fit a line, the sign of P will be determined for us.

Note: We will use both the Gutenberg-Richter and the Omori Law statistics computed in Homework 1 in Homework 2 where we will examine the probability of earthquake occurrence and aftershock occurrence following a given mainshock.


In [ ]:
#Compute the Omori Data and plot it
epsilon=0.1
maxdays=int(np.max(adays))
t=np.arange(1,maxdays+2,1)

logt=np.log10(...)
N=np.zeros(...)

#Loop to count events for each day using np.count_nonzero()
 
logN=np.log10(...)

#Code for plot here


In [ ]:
#fit a line to the logN and logt data and then compare in N-t space
p=np.polyfit(...,...,1)
logNN=np.polyval(...,...)
NN=10**logNN

#Plot comparing the data (blue dots) and model(NN) (red line)


#Print the a and P coefficients


### Questions

- Which faults were active during Loma Prieta?

- What could cause aftershocks to occur on faults other than the mainshock fault?

- How well do the Loma Prieta aftershocks follow the Omori Power Law?

- How do the results compare using method=2 to decluster the catalog?

- How well does the estimated P value compare to the range of values given in Lay and Wallace?

#### Answers here

#### Lets use the Pre-Loma Prieta daily seismicity rate to assess when the Loma Prieta sequence aftershocks end (trend into the pre-event rate)


In [ ]:
#Make a new deep copy of the LP_catalog
PRELP=LP_catalog.copy(deep=True)

#Parse catalog having the same number of months as the LPEQ catalog
start_date=pd.datetime(...)
end_date=pd.datetime(...)
PRELP=PRELP[LP_catalog['DateTime'].between(start_date,end_date)]
PRELP=PRELP.reset_index(drop=True)

year,month,day,hour,minute,sec,lat,lon,mag,days = parseCatalog(PRELP)
PRELP

In [ ]:
#First compute the number of events per day in the PRELP catalog

#Compute the mean number of earthquakes per day
meanN=np.mean(preN)
print(f'mean pre-LP events per day {meanN:.1f}')

#Use np.where() to find the index where N is less than the pre-LP daily rate.
#Use the first value of the index array returned by np.where to determine the numbers of days elaspsed when the seismicity
#rate first returns to the pre-LP average

result=np.where(...)  #N is the LP sequence daily bins, and meanN is the average from the pre-LP catalog


### Questions

1. Based on the daily rate of earthquakes when does the Loma Prieta earthquake aftershock sequence "end?"

#### Answer here

### Submission

Save the completed notebook as a `pdf` and upload to becourses.